Loading the Data

In [4]:
import pandas as pd
import numpy as np

In [5]:
# Load the data
df_0 = pd.read_csv(r"C:\Users\pingk\Downloads\fadhli nitip\asik_rt4NWIN_only CLND.csv")

Selecting Regions of Interest

In [6]:
# Define the regions of interest
regions_of_interest = [
    (2996, 3016), (2943, 2963), (2912, 2932), (2843, 2863),
    (1490, 2010), (1455, 1475), (1408, 1428), (1368, 1388),
    (1225, 1245), (1150, 1170), (1106, 1126), (1088, 1108),
    (990, 920), (712, 732)
]

# Extract columns corresponding to the regions of interest
columns_to_focus = []
for start, end in regions_of_interest:
    columns_to_focus.extend([col for col in df_0.columns[4:-5] if start <= float(col) <= end])

# Create a new DataFrame with the selected regions
df_0_selected_regions = df_0[columns_to_focus]

# Combine the selected regions with the target column and other relevant columns
df_0_selected_regions = pd.concat([df_0[['thnoth_name', 'prov_char']], df_0_selected_regions], axis=1)

# Save the DataFrame for further processing
df_0_selected_regions.to_csv('data/data file 2/data_1.csv', index=False)


FUNCTIONS

Baseline Correction

In [7]:
from scipy.signal import savgol_filter

# Function for baseline correction with dynamic window length
def baseline_correction(spectrum, default_window_length=15, polyorder=3):
    spectrum_length = len(spectrum)
    if spectrum_length < default_window_length:
        window_length = spectrum_length // 2 * 2 + 1  # Make window length odd and less than the size of the spectrum
    else:
        window_length = default_window_length
    baseline = savgol_filter(spectrum, window_length, polyorder, mode='nearest')
    corrected_spectrum = spectrum - baseline
    return corrected_spectrum

# Apply baseline correction
df_baseline_corrected_v0 = df_0_selected_regions.copy()
for col in columns_to_focus:
    df_baseline_corrected_v0[col] = baseline_correction(df_baseline_corrected_v0[col])

# Save the baseline corrected data
df_baseline_corrected_v0.to_csv('data/data file 2/data_1_bslcrct.csv', index=False)

SavGol Smoothing

In [8]:
# Function for Savitzky-Golay smoothing
def savitzky_golay_smoothing(spectrum, default_window_length=11, polyorder=2):
    window_length = min(default_window_length, len(spectrum) // 2 * 2 + 1)  # Make window length odd and less than or equal to the size of the spectrum
    if window_length < 3:  # Ensure window length is at least 3
        window_length = 3
    return savgol_filter(spectrum, window_length, polyorder, mode='nearest')  # Set mode to 'nearest'

# Apply smoothing
df_smoothed_v0 = df_baseline_corrected_v0.copy()
for col in columns_to_focus:
    df_smoothed_v0[col] = savitzky_golay_smoothing(df_smoothed_v0[col])

# Save the smoothed data
df_smoothed_v0.to_csv('data/data file 2/data_1_smoothed.csv', index=False)


Normalization

In [9]:
# Function for normalization (Min-Max scaling)
def min_max_normalization(spectrum):
    return (spectrum - np.min(spectrum)) / (np.max(spectrum) - np.min(spectrum))

# Apply normalization
df_normalized_v0 = df_smoothed_v0.copy()
for col in columns_to_focus:
    df_normalized_v0[col] = min_max_normalization(df_normalized_v0[col])

# Save the normalized data
df_normalized_v0.to_csv('data/data file 2/data_1_normalized.csv', index=False)


In [10]:
df_normalized_v0

,thnoth_name,prov_char,2996.355,2996.837,2997.32,2997.802,2998.284,2998.766,2999.248,2999.73,...,727.514,727.996,728.478,728.961,729.443,729.925,730.407,730.889,731.371,731.853
0,Thai,CBI,0.632770,0.628884,0.630682,0.637426,0.647609,0.659289,0.670642,0.680117,...,0.663686,0.627368,0.634729,0.667376,0.708908,0.728693,0.716606,0.700054,0.677500,0.646851
1,Thai,CBI,0.706441,0.704854,0.708020,0.714692,0.722795,0.730089,0.734952,0.736734,...,0.665555,0.628524,0.635070,0.666129,0.703855,0.717236,0.697969,0.675127,0.649255,0.620566
2,Thai,CBI,0.730877,0.737551,0.745549,0.753106,0.758058,0.758693,0.754514,0.746575,...,0.672184,0.604453,0.574508,0.566794,0.564563,0.548939,0.521108,0.500935,0.488252,0.482974
3,Thai,CBI,0.711660,0.718547,0.728578,0.739588,0.749019,0.754934,0.756807,0.755854,...,0.565358,0.501529,0.475699,0.470115,0.468375,0.459770,0.447565,0.445526,0.453000,0.469077
4,Thai,CBI,0.463999,0.471172,0.479789,0.488867,0.497609,0.505688,0.513274,0.521054,...,0.421279,0.387098,0.395308,0.420862,0.443505,0.451749,0.446729,0.443660,0.444498,0.449781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,Non-Thai,SUX,0.573202,0.580907,0.589288,0.597286,0.603901,0.608603,0.611456,0.613257,...,0.510228,0.460254,0.460166,0.485688,0.514746,0.528254,0.522753,0.517432,0.515444,0.520306
453,Non-Thai,SUX,0.564418,0.571421,0.580240,0.589661,0.598160,0.604422,0.607653,0.607880,...,0.380611,0.352040,0.378229,0.428608,0.478731,0.510487,0.517484,0.516899,0.511620,0.503704
454,Non-Thai,SUX,0.526169,0.534400,0.544055,0.553585,0.561322,0.566028,0.567060,0.564469,...,0.342976,0.335708,0.379948,0.444298,0.506253,0.547384,0.560112,0.563313,0.558744,0.546067
455,Non-Thai,SUX,0.465894,0.466109,0.467753,0.470157,0.472455,0.473880,0.473853,0.471911,...,0.389519,0.344744,0.350651,0.381269,0.415673,0.442661,0.458045,0.472078,0.484589,0.494524


Derivatization (np.gradient)

In [11]:
# Calculate the first derivative using np.gradient
data_spectrum = df_normalized_v0.iloc[:, 2:].values
first_derivative_np = np.gradient(data_spectrum, axis=1)

# Calculate the second derivative using np.gradient
second_derivative_np = np.gradient(first_derivative_np, axis=1)

# Convert the results back to DataFrame
data_1_der_np = pd.DataFrame(first_derivative_np, columns=df_normalized_v0.columns[2:])
data_2_der_np = pd.DataFrame(second_derivative_np, columns=df_normalized_v0.columns[2:])

# Combine the first two columns from the original dataset with the np.gradient derivatives

# Extract the first two columns
first_two_columns = df_normalized_v0.iloc[:, :2]

# Combine the first two columns with the derivatives
data_1_der_combined = pd.concat([first_two_columns, data_1_der_np], axis=1)
data_2_der_combined = pd.concat([first_two_columns, data_2_der_np], axis=1)

# Export the combined data to CSV
data_1_der_combined.to_csv('data/data file 2/data_1_1_der.csv', index=False)
data_2_der_combined.to_csv('data/data file 2/data_1_2_der.csv', index=False)

Derivatization (SavGol)

In [12]:
# Extract the spectrum data
data_spectrum = df_normalized_v0.iloc[:, 2:].values

# Apply Savitzky-Golay filter for the first derivative
first_derivative_savgol = savgol_filter(data_spectrum, window_length=5, polyorder=2, deriv=1, axis=1)

# Apply Savitzky-Golay filter for the second derivative
second_derivative_savgol = savgol_filter(data_spectrum, window_length=5, polyorder=2, deriv=2, axis=1)

# Convert the results back to DataFrame
data_1_der_savgol = pd.DataFrame(first_derivative_savgol, columns=df_normalized_v0.columns[2:])
data_2_der_savgol = pd.DataFrame(second_derivative_savgol, columns=df_normalized_v0.columns[2:])

# Extract the first two columns
first_two_columns = df_normalized_v0.iloc[:, :2]

# Combine the first two columns with the Savitzky-Golay derivatives
data_1_der_savgol_combined = pd.concat([first_two_columns, data_1_der_savgol], axis=1)
data_2_der_savgol_combined = pd.concat([first_two_columns, data_2_der_savgol], axis=1)

# Export the combined data to CSV
data_1_der_savgol_combined.to_csv('data/data file 2/data_1_1_der_savgol.csv', index=False)
data_2_der_savgol_combined.to_csv('data/data file 2/data_1_2_der_savgol.csv', index=False)

SNV

In [13]:
def snv(spectrum):
    return (spectrum - np.mean(spectrum)) / np.std(spectrum)

# Apply SNV to the selected regions
df_snv = df_0_selected_regions.copy()
for col in columns_to_focus:
    df_snv[col] = snv(df_snv[col])

# Save the SNV data
df_snv.to_csv('data/data file 2/data_1_snv.csv', index=False)


c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\numpy\_core\fromnumeric.py:3800: FutureWarning: The behavior of DataFrame.std with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return std(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)


Random Normal Variate (RNV)

In [14]:
def rnv(spectrum):
    random_noise = np.random.normal(0, np.std(spectrum), spectrum.shape)
    return spectrum + random_noise

# Apply RNV to the selected regions
df_rnv = df_0_selected_regions.copy()
for col in columns_to_focus:
    df_rnv[col] = rnv(df_rnv[col])

# Save the RNV data
df_rnv.to_csv('data/data file 2/data_1_rnv.csv', index=False)


Multiplicative Scatter Correction (MSC)

In [15]:
# def msc(spectrum, reference):
#     mean_spectrum = np.mean(reference, axis=0)
#     fit = np.polyfit(mean_spectrum, spectrum, 1, full=True)
#     corrected_spectrum = (spectrum - fit[0][1]) / fit[0][0]
#     return corrected_spectrum

# # Apply MSC to the selected regions
# df_msc = df_0_selected_regions.copy()
# for col in columns_to_focus:
#     df_msc[col] = msc(df_msc[col])

# # Save the MSC data
# df_msc.to_csv('data/data file 2/data_1_msc2.csv', index=False)


In [16]:
# Multiplicative Scatter Correction (MSC) function
def msc(input_data):
    # Mean center the data
    mean_spectrum = np.mean(input_data, axis=0)
    input_data_centered = input_data - mean_spectrum

    # Perform MSC
    reference = np.mean(input_data, axis=0)
    msc_data = np.zeros_like(input_data)

    for i in range(input_data.shape[0]):
        fit = np.polyfit(reference, input_data_centered[i, :], 1, full=True)
        msc_data[i, :] = (input_data_centered[i, :] - fit[0][1]) / fit[0][0]
    
    return msc_data

# Extract the spectral data from the dataframe
spectral_data = df_0_selected_regions.iloc[:, 2:].values

# Apply MSC
msc_corrected_data = msc(spectral_data)

# Create a new dataframe with the MSC corrected data
msc_df = pd.DataFrame(msc_corrected_data, columns=df_0_selected_regions.columns[2:])
msc_df.insert(0, 'prov_char', df_0_selected_regions['prov_char'])
msc_df.insert(0, 'thnoth_name', df_0_selected_regions['thnoth_name'])

# Save the MSC corrected data to a new CSV file
msc_corrected_file_path = 'data/data file 2/data_1_msc_corrected.csv'
msc_df.to_csv(msc_corrected_file_path, index=False)


Classification and Evaluation (40-fold)

In [17]:
# Define the target variable
target = 'thnoth_name'

# Ensure columns_to_focus are correctly identified
numeric_cols_df_0 = df_0_selected_regions.select_dtypes(include=[np.number]).columns.tolist()
columns_to_focus = numeric_cols_df_0  # Ensure columns are correctly selected

# Classification and evaluation function using 40-fold CV
def classify_and_evaluate(df, columns):
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import cross_val_score
    from sklearn.preprocessing import LabelEncoder

    # Encode target variable
    le = LabelEncoder()
    y = le.fit_transform(df[target])

    # Define features
    X = df[columns]

    # Initialize the classifier
    model = RandomForestClassifier(n_estimators=100, random_state=42)

    # Perform cross-validation
    scores = cross_val_score(model, X, y, cv=40)

    # Print the results
    print(f'Cross-Validation Accuracy: {np.mean(scores)}')


In [18]:

# Evaluate each preprocessing method
print("Evaluation for No Preprocessing:")
classify_and_evaluate(df_0_selected_regions, columns_to_focus)

Evaluation for No Preprocessing:
Cross-Validation Accuracy: 0.8873106060606061


In [19]:
print("Evaluation for Baseline Correction:")
df_baseline_corrected_v0z = pd.read_csv('data/data file 2/data_1_bslcrct.csv')
classify_and_evaluate(df_baseline_corrected_v0z, columns_to_focus)

Evaluation for Baseline Correction:
Cross-Validation Accuracy: 0.7339015151515151


In [20]:
print("Evaluation for Smoothing:")
df_smoothed_v0z = pd.read_csv('data/data file 2/data_1_smoothed.csv')
classify_and_evaluate(df_smoothed_v0z, columns_to_focus)

Evaluation for Smoothing:
Cross-Validation Accuracy: 0.7759469696969697


In [21]:
print("Evaluation for Normalization:")
df_normalized_v0z = pd.read_csv('data/data file 2/data_1_normalized.csv')
classify_and_evaluate(df_normalized_v0z, columns_to_focus)

Evaluation for Normalization:
Cross-Validation Accuracy: 0.7638257575757577


In [22]:
print("Evaluation for 1-Derivative Spectroscopy:")
data_1_der_combined_v0z = pd.read_csv('data/data file 2/data_1_1_der.csv')
classify_and_evaluate(data_1_der_combined_v0z, columns_to_focus)

Evaluation for 1-Derivative Spectroscopy:
Cross-Validation Accuracy: 0.7668560606060606


In [23]:
print("Evaluation for 2-Derivative Spectroscopy:")
data_2_der_combined_v0z = pd.read_csv('data/data file 2/data_1_2_der.csv')
classify_and_evaluate(data_2_der_combined_v0z, columns_to_focus)

Evaluation for 2-Derivative Spectroscopy:
Cross-Validation Accuracy: 0.7893939393939394


In [24]:
print("Evaluation for 1-SG-Derivative Spectroscopy:")
data_1_der_savgol_combined_v0z = pd.read_csv('data/data file 2/data_1_1_der_savgol.csv')
classify_and_evaluate(data_1_der_savgol_combined_v0z, columns_to_focus)

Evaluation for 1-SG-Derivative Spectroscopy:
Cross-Validation Accuracy: 0.7880681818181818


In [25]:
print("Evaluation for 2-SG-Derivative Spectroscopy:")
data_1_der_savgol_combined_v0z = pd.read_csv('data/data file 2/data_1_2_der_savgol.csv')
classify_and_evaluate(data_1_der_savgol_combined_v0z, columns_to_focus)

Evaluation for 2-SG-Derivative Spectroscopy:
Cross-Validation Accuracy: 0.7746212121212122


In [27]:
print("Evaluation for SNV:")
df_snv_v0z = pd.read_csv('data/data file 2/data_1_snv.csv')
classify_and_evaluate(df_snv_v0z, columns_to_focus)

print("Evaluation for RNV:")
df_rnv_v0z = pd.read_csv('data/data file 2/data_1_rnv.csv')
classify_and_evaluate(df_rnv_v0z, columns_to_focus)

print("Evaluation for MSC:")
df_msc_v0z = pd.read_csv('data/data file 2/data_1_msc_corrected.csv')
classify_and_evaluate(df_msc_v0z, columns_to_focus)

Evaluation for SNV:
Cross-Validation Accuracy: 0.8827651515151516
Evaluation for RNV:
Cross-Validation Accuracy: 0.7888257575757576
Evaluation for MSC:
Cross-Validation Accuracy: 0.8660984848484847


Classification and Evaluation (40-fold) -with extra detailed

In [28]:
# Define the target variable
target = 'thnoth_name'

# Ensure columns_to_focus are correctly identified
numeric_cols_df_0 = df_0_selected_regions.select_dtypes(include=[np.number]).columns.tolist()
columns_to_focus = numeric_cols_df_0  # Ensure columns are correctly selected

# Classification and evaluation function using 40-fold CV with detailed metrics
def classify_and_evaluate(df, columns):
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import cross_val_score, StratifiedKFold
    from sklearn.preprocessing import LabelEncoder
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
    import numpy as np

    # Encode target variable
    le = LabelEncoder()
    y = le.fit_transform(df[target])

    # Define features
    X = df[columns]

    # Initialize the classifier
    model = RandomForestClassifier(n_estimators=100, random_state=42)

    # Initialize Stratified K-Fold Cross-Validation
    skf = StratifiedKFold(n_splits=40)

    # Arrays to store results
    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []
    conf_matrices = []

    # Perform cross-validation
    for train_idx, test_idx in skf.split(X, y):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # Train the model
        model.fit(X_train, y_train)

        # Predict and evaluate
        y_pred = model.predict(X_test)
        accuracies.append(accuracy_score(y_test, y_pred))
        precisions.append(precision_score(y_test, y_pred, average=None))
        recalls.append(recall_score(y_test, y_pred, average=None))
        f1_scores.append(f1_score(y_test, y_pred, average=None))
        conf_matrices.append(confusion_matrix(y_test, y_pred))

    # Calculate mean scores
    mean_accuracy = np.mean(accuracies)
    mean_precision = np.mean(precisions, axis=0)
    mean_recall = np.mean(recalls, axis=0)
    mean_f1 = np.mean(f1_scores, axis=0)
    mean_conf_matrix = np.mean(conf_matrices, axis=0)

    # Print the results
    print(f'Cross-Validation Accuracy: {mean_accuracy}')
    print(f'Precision per class: {mean_precision}')
    print(f'Recall per class: {mean_recall}')
    print(f'F1-score per class: {mean_f1}')


In [29]:
# Evaluate each preprocessing method
print("Evaluation for No Preprocessing:")
classify_and_evaluate(df_0_selected_regions, columns_to_focus)

print("Evaluation for Baseline Correction:")
df_baseline_corrected_v0y = pd.read_csv('data/data file 2/data_1_bslcrct.csv')
classify_and_evaluate(df_baseline_corrected_v0y, columns_to_focus)

print("Evaluation for Smoothing:")
df_smoothed_v0y = pd.read_csv('data/data file 2/data_1_smoothed.csv')
classify_and_evaluate(df_smoothed_v0y, columns_to_focus)

print("Evaluation for Normalization:")
df_normalized_v0y = pd.read_csv('data/data file 2/data_1_normalized.csv')
classify_and_evaluate(df_normalized_v0y, columns_to_focus)

print("Evaluation for 1-Derivative Spectroscopy:")
df_derivative_v0y = pd.read_csv('data/data file 2/data_1_1_der.csv')
classify_and_evaluate(df_derivative_v0y, columns_to_focus)

print("Evaluation for 2-Derivative Spectroscopy:")
data_2_der_combined_v0y = pd.read_csv('data/data file 2/data_1_2_der.csv')
classify_and_evaluate(data_2_der_combined_v0y, columns_to_focus)

print("Evaluation for 1-SG-Derivative Spectroscopy:")
data_1_der_savgol_combined_v0y = pd.read_csv('data/data file 2/data_1_1_der_savgol.csv')
classify_and_evaluate(data_1_der_savgol_combined_v0y, columns_to_focus)

print("Evaluation for 2-SG-Derivative Spectroscopy:")
data_1_der_savgol_combined_v0y = pd.read_csv('data/data file 2/data_1_2_der_savgol.csv')
classify_and_evaluate(data_1_der_savgol_combined_v0y, columns_to_focus)

Evaluation for No Preprocessing:


c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicte

Cross-Validation Accuracy: 0.8873106060606061
Precision per class: [0.84416667 0.88837753]
Recall per class: [0.69583333 0.984375  ]
F1-score per class: [0.7314881  0.92727846]
Evaluation for Baseline Correction:


c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicte

Cross-Validation Accuracy: 0.7339015151515151
Precision per class: [0.34291667 0.74756674]
Recall per class: [0.23333333 0.95803571]
F1-score per class: [0.25936508 0.83452571]
Evaluation for Smoothing:


...

Cross-Validation Accuracy: 0.7893939393939394
Precision per class: [0.59791667 0.78632305]
Recall per class: [0.36041667 0.98392857]
F1-score per class: [0.42416667 0.8690854 ]
Evaluation for 1-SG-Derivative Spectroscopy:


c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicte

Cross-Validation Accuracy: 0.7880681818181818
Precision per class: [0.50125    0.80572601]
Recall per class: [0.43541667 0.95267857]
F1-score per class: [0.44950397 0.86346785]
Evaluation for 2-SG-Derivative Spectroscopy:


c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicte

Cross-Validation Accuracy: 0.7746212121212122
Precision per class: [0.53125    0.77183081]
Recall per class: [0.31875    0.98080357]
F1-score per class: [0.38577381 0.86013975]


In [30]:
print("Evaluation for SNV:")
df_snv_v0y = pd.read_csv('data/data file 2/data_1_snv.csv')
classify_and_evaluate(df_snv_v0y, columns_to_focus)

print("Evaluation for RNV:")
df_rnv_v0y = pd.read_csv('data/data file 2/data_1_rnv.csv')
classify_and_evaluate(df_rnv_v0y, columns_to_focus)

print("Evaluation for MSC:")
df_msc_v0y = pd.read_csv('data/data file 2/data_1_msc_corrected.csv')
classify_and_evaluate(df_msc_v0y, columns_to_focus)

Evaluation for SNV:


c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicte

Cross-Validation Accuracy: 0.8827651515151516
Precision per class: [0.80875    0.88700126]
Recall per class: [0.68958333 0.98125   ]
F1-score per class: [0.71625    0.92459184]
Evaluation for RNV:


c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicte

Cross-Validation Accuracy: 0.7888257575757576
Precision per class: [0.48333333 0.79170455]
Recall per class: [0.37916667 0.98125   ]
F1-score per class: [0.4122619  0.86996281]
Evaluation for MSC:


c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicte

Cross-Validation Accuracy: 0.8660984848484847
Precision per class: [0.82940476 0.88078914]
Recall per class: [0.67708333 0.95625   ]
F1-score per class: [0.70987554 0.90840794]


c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Classification and Evaluation using LOGO-CV

In [31]:
# Classification and evaluation function using LOGO-CV with detailed metrics
def classify_and_evaluate_logo_cv_detailed(df, columns):
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import LeaveOneGroupOut
    from sklearn.preprocessing import LabelEncoder
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
    
    # Encode target variable
    le = LabelEncoder()
    y = le.fit_transform(df[target])
    
    # Define features
    X = df[columns]
    
    # Initialize the classifier
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    
    # Initialize LOGO-CV
    logo = LeaveOneGroupOut()
    groups = df['prov_char']
    
    # Arrays to store results
    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []
    all_y_test = []
    all_y_pred = []
    
    # Perform LOGO-CV
    for train_idx, test_idx in logo.split(X, y, groups=groups):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        # Train the model
        model.fit(X_train, y_train)
        
        # Predict and evaluate
        y_pred = model.predict(X_test)
        accuracies.append(accuracy_score(y_test, y_pred))
        all_y_test.extend(y_test)
        all_y_pred.extend(y_pred)
        
    # Calculate overall metrics
    mean_accuracy = np.mean(accuracies)
    precision = precision_score(all_y_test, all_y_pred, average=None)
    recall = recall_score(all_y_test, all_y_pred, average=None)
    f1 = f1_score(all_y_test, all_y_pred, average=None)
    conf_matrix = confusion_matrix(all_y_test, all_y_pred)
    
    # Print the results
    print(f'Mean Accuracy: {mean_accuracy}')
    print(f'Precision per class: {precision}')
    print(f'Recall per class: {recall}')
    print(f'F1-score per class: {f1}')
    
    # Return confusion matrix for presentation
    return conf_matrix

In [32]:
# Evaluate each preprocessing method using LOGO-CV with detailed metrics
print("LOGO-CV Evaluation for No Preprocessing:")
conf_matrix_no_preprocessing_v0x = classify_and_evaluate_logo_cv_detailed(df_0_selected_regions, columns_to_focus)

print("LOGO-CV Evaluation for Baseline Correction:")
df_baseline_corrected_v0x = pd.read_csv('data/data file 2/data_1_bslcrct.csv')
conf_matrix_baseline_v0x = classify_and_evaluate_logo_cv_detailed(df_baseline_corrected_v0x, columns_to_focus)

print("LOGO-CV Evaluation for Smoothing:")
df_smoothed_v0x = pd.read_csv('data/data file 2/data_1_smoothed.csv')
conf_matrix_smoothing_v0x = classify_and_evaluate_logo_cv_detailed(df_smoothed_v0x, columns_to_focus)

print("LOGO-CV Evaluation for Normalization:")
df_normalized_v0x = pd.read_csv('data/data file 2/data_1_normalized.csv')
conf_matrix_normalization_v0x = classify_and_evaluate_logo_cv_detailed(df_normalized_v0x, columns_to_focus)

print("LOGO-CV Evaluation for 1-Derivative Spectroscopy:")
df_1_derivative_v0x = pd.read_csv('data/data file 2/data_1_1_der.csv')
conf_matrix_1_derivative_v0x = classify_and_evaluate_logo_cv_detailed(df_1_derivative_v0x, columns_to_focus)

print("LOGO-CV Evaluation for 2-Derivative Spectroscopy:")
df_2_derivative_v0x = pd.read_csv('data/data file 2/data_1_2_der.csv')
conf_matrix_2_derivative_v0x = classify_and_evaluate_logo_cv_detailed(df_2_derivative_v0x, columns_to_focus)

print("LOGO-CV Evaluation for 1-SG-Derivative Spectroscopy:")
df_1_der_savgol_combined_v0y = pd.read_csv('data/data file 2/data_1_1_der_savgol.csv')
conf_matrix_1_sg_v0x = classify_and_evaluate_logo_cv_detailed(df_1_der_savgol_combined_v0y, columns_to_focus)

print("LOGO-CV Evaluation for 2-SG-Derivative Spectroscopy:")
df_2_der_savgol_combined_v0y = pd.read_csv('data/data file 2/data_1_2_der_savgol.csv')
conf_matrix_2_sg_v0x = classify_and_evaluate_logo_cv_detailed(df_2_der_savgol_combined_v0y, columns_to_focus)

print("LOGO-CV Evaluation for SNV:")
df_snv_v0x = pd.read_csv('data/data file 2/data_1_snv.csv')
conf_matrix_snv_v0x = classify_and_evaluate_logo_cv_detailed(df_snv_v0x, columns_to_focus)

print("LOGO-CV Evaluation for RNV:")
df_rnv_v0x = pd.read_csv('data/data file 2/data_1_rnv.csv')
conf_matrix_rnv_v0x = classify_and_evaluate_logo_cv_detailed(df_rnv_v0x, columns_to_focus)

print("LOGO-CV Evaluation for MSC:")
df_msc_v0x = pd.read_csv('data/data file 2/data_1_msc_corrected.csv')
conf_matrix_msc_v0x = classify_and_evaluate_logo_cv_detailed(df_msc_v0x, columns_to_focus)

LOGO-CV Evaluation for No Preprocessing:
Mean Accuracy: 0.8000928983255498
Precision per class: [0.78666667 0.78272251]
Recall per class: [0.41549296 0.94920635]
F1-score per class: [0.5437788 0.8579627]
LOGO-CV Evaluation for Baseline Correction:
Mean Accuracy: 0.7256274479933755
Precision per class: [0.60416667 0.72371638]
Recall per class: [0.20422535 0.93968254]
F1-score per class: [0.30526316 0.81767956]
LOGO-CV Evaluation for Smoothing:
Mean Accuracy: 0.7294378066539143
Precision per class: [0.64       0.72972973]
Recall per class: [0.22535211 0.94285714]
F1-score per class: [0.33333333 0.82271468]
LOGO-CV Evaluation for Normalization:
Mean Accuracy: 0.7505995846166074
Precision per class: [0.72       0.73955774]
Recall per class: [0.25352113 0.95555556]
F1-score per class: [0.375      0.83379501]
LOGO-CV Evaluation for 1-Derivative Spectroscopy:
Mean Accuracy: 0.7384807183930872
Precision per class: [0.64864865 0.71904762]
Recall per class: [0.16901408 0.95873016]
F1-score per c

In [ ]:
conf_matrix_1_sg_v0x = classify_and_evaluate_logo_cv_detailed(df_1_der_savgol_combined_v0y, columns_to_focus)

Mean Accuracy: 0.9194485925414478
Precision per class: [0.93984962 0.84210526]
Recall per class: [0.82236842 0.94736842]
F1-score per class: [0.87719298 0.89164087]


Display Confusion Matrices

In [33]:
# Function to display confusion matrix in a tabular format
def display_confusion_matrix(conf_matrix, class_labels):
    df_cm = pd.DataFrame(conf_matrix, index=class_labels, columns=class_labels)
    print(df_cm)

# Ensure LabelEncoder is defined and class labels are set
from sklearn.preprocessing import LabelEncoder

# Define the target variable and fit LabelEncoder
target = 'thnoth_name'
le = LabelEncoder()
le.fit(df_0_selected_regions[target])
class_labels = le.classes_

In [34]:
# Display confusion matrices for each preprocessing method
print("Confusion Matrix for No Preprocessing:")
display_confusion_matrix(conf_matrix_no_preprocessing_v0x, class_labels)

print("Confusion Matrix for Baseline Correction:")
display_confusion_matrix(conf_matrix_baseline_v0x, class_labels)

print("Confusion Matrix for Smoothing:")
display_confusion_matrix(conf_matrix_smoothing_v0x, class_labels)

print("Confusion Matrix for Normalization:")
display_confusion_matrix(conf_matrix_normalization_v0x, class_labels)

print("Confusion Matrix for Derivative Spectroscopy:")
display_confusion_matrix(conf_matrix_1_derivative_v0x, class_labels)

print("Confusion Matrix for Derivative Spectroscopy:")
display_confusion_matrix(conf_matrix_2_derivative_v0x, class_labels)

print("Confusion Matrix for Derivative Spectroscopy:")
display_confusion_matrix(conf_matrix_1_sg_v0x, class_labels)

print("Confusion Matrix for Derivative Spectroscopy:")
display_confusion_matrix(conf_matrix_2_sg_v0x, class_labels)

print("Confusion Matrix for SNV:")
display_confusion_matrix(conf_matrix_snv_v0x, class_labels)

print("Confusion Matrix for RNV:")
display_confusion_matrix(conf_matrix_rnv_v0x, class_labels)

print("Confusion Matrix for MSC:")
display_confusion_matrix(conf_matrix_msc_v0x, class_labels)

Confusion Matrix for No Preprocessing:
          Non-Thai  Thai
Non-Thai        59    83
Thai            16   299
Confusion Matrix for Baseline Correction:
          Non-Thai  Thai
Non-Thai        29   113
Thai            19   296
Confusion Matrix for Smoothing:
          Non-Thai  Thai
Non-Thai        32   110
Thai            18   297
Confusion Matrix for Normalization:
          Non-Thai  Thai
Non-Thai        36   106
Thai            14   301
Confusion Matrix for Derivative Spectroscopy:
          Non-Thai  Thai
Non-Thai        24   118
Thai            13   302
Confusion Matrix for Derivative Spectroscopy:
          Non-Thai  Thai
Non-Thai        24   118
Thai            14   301
Confusion Matrix for Derivative Spectroscopy:
          Non-Thai  Thai
Non-Thai        31   111
Thai            16   299
Confusion Matrix for Derivative Spectroscopy:
          Non-Thai  Thai
Non-Thai        22   120
Thai            16   299
Confusion Matrix for SNV:
          Non-Thai  Thai
Non-Thai        